# Voltage 2

M&I Ch 16 Electric Potential

- § 16.3 &ndash; Potential Difference in a Uniform Field
- § 16.6 &ndash; Path Independence

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   {{ graphic }}
>
>   Locations $A$ and $B$ are in a region of uniform electric field, as shown. 
>
>   Along a path from $A$ to $B$, the change in potential is $2900$ [V].
>   The distance from $A$ to $B$ is $0.22$ [m]
>
>   __What is the magnitude of the electric field in this region, with units?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\lvert \vec{E} \rvert = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [V/m]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]*
>
>   If the electric field exceeds about $3 \times 10^6$ [N/C] in air, a 
>   spark occurs. Approximately, __what is the maximum possible potential 
>   difference between the plates of a capacitor__ whose gap is $3$ [mm],
>   without causing a spark in the air between them?
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   In an old style television picture tube (not in a modern flat-panel TV),
>   electrons are boiled out of a very hot metal filament placed near a negative
>   metal plate (see figure).  
>
>   {{ graphic }}
>
>   These electrons start out nearly at rest, and are accelerated toward a 
>   positive metal plate. They pass through a hole in the positive plate on 
>   their way toward the picture screen. 
>
>   If the high-voltage supply in the television set maintains a potential
>   difference of 15,500 volts between the two plates, __what speed do the 
>   electrons reach?__
>
>   (You can use the non-relativisitic approximation here.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $v = \fbox{\hspace{8em}?}$ &ensp; [m/s]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]*
>
>   A capacitor consists of two charged disks of radius $R =4.0$ [m], 
>   separated by a distance $s = 2$ [mm] (see figure below).
>
>   {{ graphic }}
>
>   The magnitude of the charge on each disk is $47$ [µC].
>
>   Consider points $A$, $B$, $C$, and $D$ inside the capacitor, as shown in 
>   the diagram. 
>
>   The distance $s_1 = 1.6$ [mm], and the distance $s_2 = 0.5$ [mm].
>
>   (Assume the $+x$-axis to the right, $+y$-axis is up, and the $+z$-axis
>    is out.)
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the electric field inside the capacitor?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N}/\mathrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   First, calculate the potential difference $V_B - V_A$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_B - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_B - V_A = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(C)
>
>   Next, calculate the potential difference $V_C - V_B$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_B - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_B = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(D)
>
>   Finally, calculate the potential difference $V_A - V_C$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_A - V_C$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_A - V_C = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   A capacitor consists of two charged disks of radius $R = 4.3$ [m], separated
>   by a distance $s = 2$ [mm] (see figure).
>
>   {{ graphic }}
>
>   The magnitude of the charge on each disk is $64$ [µC].
>
>   Consider points $A$, $B$, $C$, and $D$ inside the capacitor, as shown in the
>   diagram. 
>
>   The distance $s_1 = 1.6$ [mm], and the distance $s_2 = 0.5$ [mm].
>
>   We will calculate the potential difference $\Delta V = V_C - V_A$ along two
>   different paths. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='5a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the magnitude of the electric field inside the capacitor?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert E \rvert = \fbox{\hspace{8em}?}\ \left[\mathrm{N}/\mathrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(B)
>
>   First, consider the path $A$-$B$-$C$:
>
>   __What is $V_B - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_B - V_A = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_B = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_A = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(C)
>
>   Now consider the path $A$-$C$. 
>
>   Along this path, $\Delta V = E \Delta L \cos(\theta)$
>
>   __If $\theta$ is the angle between $\vec{E}$ and the displacement $\Delta 
>   \vec{L}$, then what is $\Delta L\ \cos\left(\theta\right)$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta L\ \cos\theta = \fbox{\hspace{8em}?}\ \left[\mathrm{m}\right]$
>
>   __Therefore, what is $\Delta V$ along the diagonal path from $A$ to $C$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_A = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---